# Алгоритм Решающее дерево

## Задача классификации

In [49]:
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [50]:
path = 'winequality-red.csv'
data_class = pd.read_csv(path)

print("Первые 5 строк датасета для классификации:")
print(data_class.head())
print(f"\nРазмер датасета: {data_class.shape}")


Первые 5 строк датасета для классификации:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  

### Создаем целевую переменную для классификации

In [51]:
data_class['wine_quality'] = data_class['quality'].apply(lambda x: 'good' if x >= 6 else 'bad')
data_class = data_class.drop('quality', axis=1)

print("\nРаспределение качества вина:")
print(data_class['wine_quality'].value_counts())


Распределение качества вина:
wine_quality
good    855
bad     744
Name: count, dtype: int64


### Разделим датасет на признаки и целевую переменную, масштабирование признаков

In [52]:
X = data_class.drop('wine_quality', axis=1)
y = data_class['wine_quality']

# масштабирование 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (1279, 11), Размер тестовой выборки: (320, 11)


#### Применим встроенное решающее дерево для классификации

In [53]:
dt_baseline = DecisionTreeClassifier(random_state=42)
dt_baseline.fit(X_train, y_train)

y_pred = dt_baseline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Бейзлайн (решающее дерево):")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Бейзлайн (решающее дерево):
Accuracy: 0.7562
F1-Score: 0.7561

Classification Report:
              precision    recall  f1-score   support

         bad       0.74      0.73      0.74       149
        good       0.77      0.78      0.77       171

    accuracy                           0.76       320
   macro avg       0.76      0.75      0.75       320
weighted avg       0.76      0.76      0.76       320



#### Подберем гиперпараметры для оптимизации бейзлайна

In [54]:
param_grid = {
    'max_depth': range(3, 21),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(1, 10)
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Лучшие параметры: {best_params}")

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Лучшие параметры: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}


#### Обучим модель с этими гиперпараметрами

In [55]:
best_dt = grid_search.best_estimator_

y_pred_best = best_dt.predict(X_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best, average='weighted')

print("Улучшенный бейзлайн (решающее дерево):")
print(f"Accuracy: {accuracy_best:.4f}")
print(f"F1-Score: {f1_best:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

Улучшенный бейзлайн (решающее дерево):
Accuracy: 0.7594
F1-Score: 0.7589

Classification Report:
              precision    recall  f1-score   support

         bad       0.75      0.72      0.74       149
        good       0.76      0.80      0.78       171

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



#### Собственная имплементация алгоритма

In [56]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class CustomDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        unique_labels = np.unique(y)

        # Добавляем проверку min_samples_leaf
        if (len(unique_labels) == 1 or 
            depth >= self.max_depth or 
            n_samples < self.min_samples_split or
            n_samples < 2 * self.min_samples_leaf):  # Нельзя разделить если samples < 2*min_samples_leaf
            most_common_label = Counter(y).most_common(1)[0][0]
            return DecisionTreeNode(value=most_common_label)

        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            most_common_label = Counter(y).most_common(1)[0][0]
            return DecisionTreeNode(value=most_common_label)

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold
        
        # Проверяем min_samples_leaf для дочерних узлов
        if sum(left_indices) < self.min_samples_leaf or sum(right_indices) < self.min_samples_leaf:
            most_common_label = Counter(y).most_common(1)[0][0]
            return DecisionTreeNode(value=most_common_label)

        left_child = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        return DecisionTreeNode(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)


    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_gain = -1
        split_idx, split_threshold = None, None

        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self._information_gain(y, X[:, feature_idx], threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = threshold

        return split_idx, split_threshold

    def _information_gain(self, y, feature_column, threshold):
        parent_loss = self._gini(y)

        left_indices = feature_column <= threshold
        right_indices = feature_column > threshold
        if sum(left_indices) == 0 or sum(right_indices) == 0:
            return 0

        n = len(y)
        n_left, n_right = sum(left_indices), sum(right_indices)
        e_left, e_right = self._gini(y[left_indices]), self._gini(y[right_indices])

        child_loss = (n_left / n) * e_left + (n_right / n) * e_right
        return parent_loss - child_loss

    def _gini(self, y):
        proportions = [np.sum(y == c) / len(y) for c in np.unique(y)]
        return 1 - sum([p ** 2 for p in proportions])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

custom_dt = CustomDecisionTree()
custom_dt.fit(X_train, y_train)

y_pred_custom = custom_dt.predict(X_test)

accuracy_custom = accuracy_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom, average='weighted')

print("Реализация Custom Decision Tree:")
print(f"Accuracy: {accuracy_custom:.4f}")
print(f"F1-Score: {f1_custom:.4f}")

custom_dt_improved = CustomDecisionTree(
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=1  # можно тоже подбирать
)
custom_dt_improved.fit(X_train, y_train)
y_pred_custom_improved = custom_dt_improved.predict(X_test)

accuracy_custom_improved = accuracy_score(y_test, y_pred_custom_improved)
f1_custom_improved = f1_score(y_test, y_pred_custom_improved, average='weighted')

print("Custom Decision Tree с улучшенными параметрами:")
print(f"Accuracy: {accuracy_custom_improved:.4f}")
print(f"F1-Score: {f1_custom_improved:.4f}")

Реализация Custom Decision Tree:
Accuracy: 0.7188
F1-Score: 0.7188
Custom Decision Tree с улучшенными параметрами:
Accuracy: 0.7531
F1-Score: 0.7528


#### Сравнение результатов

In [57]:
print("\nСравнение результатов для задачи классификации:")
print(f"Бейзлайн Accuracy: {accuracy:.4f}, Улучшенный Accuracy: {accuracy_best:.4f}, Custom Decision Tree Accuracy: {accuracy_custom:.4f}, Улучшенный Custom Decision Tree Accuracy: {accuracy_custom_improved:.4f}")
print(f"Бейзлайн F1-Score: {f1:.4f}, Улучшенный F1-Score: {f1_best:.4f}, Custom Decision Tree F1-Score: {f1_custom:.4f}, Улучшенный Custom Decision Tree F1-Score: {f1_custom_improved:.4f}")


Сравнение результатов для задачи классификации:
Бейзлайн Accuracy: 0.7562, Улучшенный Accuracy: 0.7594, Custom Decision Tree Accuracy: 0.7188, Улучшенный Custom Decision Tree Accuracy: 0.7531
Бейзлайн F1-Score: 0.7561, Улучшенный F1-Score: 0.7589, Custom Decision Tree F1-Score: 0.7188, Улучшенный Custom Decision Tree F1-Score: 0.7528


## Задача регрессии

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [59]:
path = 'hour.csv'
data_reg = pd.read_csv(path)

print("\nПервые 5 строк датасета для регрессии:")
print(data_reg.head())

# убрал дату в строковом представлении (есть дата в числовых колонках), так же число, которое предсказываем (cnt)
useful_columns = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 
                  'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'registered']

# Разделим данные на тестовую и обучающую выборку
X = data_reg[useful_columns]
y = data_reg['cnt']

print(f"\nИспользуемые признаки: {list(X.columns)}")
print(f"Размер X: {X.shape}")


Первые 5 строк датасета для регрессии:
   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  

Используемые признаки: ['season', 'yr', 'mnth', 'hr', 'hol

#### Разделим данные

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (13903, 13), Размер тестовой выборки: (3476, 13)


#### Масштабирование признаков

In [61]:
scaler_reg = StandardScaler()
X_train_scaled = scaler_reg.fit_transform(X_train)
X_test_scaled = scaler_reg.transform(X_test)

#### Обучение бейзлайна и оценка модели

In [62]:
dt_baseline = DecisionTreeRegressor(random_state=42)
dt_baseline.fit(X_train_scaled, y_train)
y_pred_reg = dt_baseline.predict(X_test_scaled)

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

print("\nБейзлайн (Decision Tree Regressor):")
mae_baseline, mse_baseline, r2_baseline = evaluate_model(y_test, y_pred_reg)
print(f"MAE: {mae_baseline:.4f}, MSE: {mse_baseline:.4f}, R^2: {r2_baseline:.4f}")


Бейзлайн (Decision Tree Regressor):
MAE: 13.4833, MSE: 587.1974, R^2: 0.9815


#### Подберем гиперпараметры для улучшения модели

In [63]:
param_grid_extended = {
    'max_depth': range(1, 20),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(1, 10),      # Минимальные samples в листе
    'max_features': [None, 'sqrt', 'log2'] # Количество признаков для разделения
}

grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="r2",
    verbose=1,
)
grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
print(f"\nЛучшие параметры: {best_params}")

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


KeyboardInterrupt: 

#### Обучим модели с подобранными гиперпараметрами

In [ ]:
dt_optimized = DecisionTreeRegressor(**best_params, random_state=42)
dt_optimized.fit(X_train_scaled, y_train)
y_pred_optimized = dt_optimized.predict(X_test_scaled)

mae_optimized, mse_optimized, r2_optimized = evaluate_model(y_test, y_pred_optimized)

print("\nУлучшенная модель (Decision Tree Regressor):")
print(f"MAE: {mae_optimized:.4f}, MSE: {mse_optimized:.4f}, R^2: {r2_optimized:.4f}")


Улучшенная модель (Decision Tree Regressor):
MAE: 11.8963, MSE: 454.4274, R^2: 0.9856


#### Реализуем собственную имплементацию алгоритма

In [ ]:
class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def predict(self, X):
        return np.array([self._predict_sample(self.tree, x) for x in X])

    def _build_tree(self, X, y, depth):
        if (depth >= self.max_depth or 
            len(np.unique(y)) == 1 or 
            len(y) < self.min_samples_split):
            return np.mean(y)

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None:
            return np.mean(y)

        left_mask = X[:, best_feature] < best_threshold
        right_mask = ~left_mask

        # Проверяем минимальное количество samples
        if sum(left_mask) < 1 or sum(right_mask) < 1:
            return np.mean(y)

        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {"feature": best_feature, "threshold": best_threshold, "left": left_child, "right": right_child}


    def _find_best_split(self, X, y):
        best_feature, best_threshold, best_mse = None, None, float("inf")
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] < threshold
                right_mask = ~left_mask

                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                mse = (
                    len(y[left_mask]) * np.var(y[left_mask]) +
                    len(y[right_mask]) * np.var(y[right_mask])
                )

                if mse < best_mse:
                    best_feature, best_threshold, best_mse = feature, threshold, mse

        return best_feature, best_threshold

    def _predict_sample(self, tree, x):
        if not isinstance(tree, dict):
            return tree

        if x[tree["feature"]] < tree["threshold"]:
            return self._predict_sample(tree["left"], x)
        else:
            return self._predict_sample(tree["right"], x)

custom_dt_reg = CustomDecisionTreeRegressor()
custom_dt_reg.fit(X_train_scaled, y_train)
y_pred_custom_reg = custom_dt_reg.predict(X_test_scaled)

mae_custom, mse_custom, r2_custom = evaluate_model(y_test, y_pred_custom_reg)

print("\nCustom Decision Tree Regressor:")
print(f"MAE: {mae_custom:.4f}, MSE: {mse_custom:.4f}, R^2: {r2_custom:.4f}")

custom_dt_reg_improved = CustomDecisionTreeRegressorImproved(
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"]
)
custom_dt_reg_improved.fit(X_train_scaled, y_train)
y_pred_custom_reg_improved = custom_dt_reg_improved.predict(X_test_scaled)

mae_custom_improved, mse_custom_improved, r2_custom_improved = evaluate_model(y_test, y_pred_custom_reg_improved)

print("Custom Decision Tree Regressor с улучшенными параметрами:")
print(f"MAE: {mae_custom_improved:.4f}, MSE: {mse_custom_improved:.4f}, R²: {r2_custom_improved:.4f}")


Custom Decision Tree Regressor:
MAE: 13.1441, MSE: 558.5635, R^2: 0.9824


#### Сравним полученные результаты

In [ ]:
print("\nСравнение результатов для задачи регрессии:")
print(f"Бейзлайн MAE: {mae_baseline:.4f}, Улучшенный MAE: {mae_optimized:.4f}, Custom MAE: {mae_custom:.4f}, Улучшенный Custom MAE: {mae_custom_improved:.4f}")
print(f"Бейзлайн MSE: {mse_baseline:.4f}, Улучшенный MSE: {mse_optimized:.4f}, Custom MSE: {mse_custom:.4f}, Улучшенный Custom MSE: {mse_custom_improved:.4f}")
print(f"Бейзлайн R^2: {r2_baseline:.4f}, Улучшенный R^2: {r2_optimized:.4f}, Custom R^2: {r2_custom:.4f} , Улучшенный Custom R^2: {r2_custom_improved:.4f}")


Сравнение результатов для задачи регрессии:
Бейзлайн MAE: 13.4833, Улучшенный MAE: 11.8963, Custom MAE: 13.1441
Бейзлайн MSE: 587.1974, Улучшенный MSE: 454.4274, Custom MSE: 558.5635
Бейзлайн R^2: 0.9815, Улучшенный R^2: 0.9856, Custom R^2: 0.9824
